In [ ]:
# Function to predict next N days price based on past T days.
# 3 years data for training
# News extraction using Beatiful soup from Business standard (only for educational demonstration purpose, not to be used for any commerical purpose)
# Stock price extraction using pandas data reader from yahoo finance

In [2]:
# import libraries
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)
import pandas_datareader.data as web 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
np.set_printoptions(suppress=True)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import sys
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from math import isnan
from sklearn.metrics import mean_absolute_percentage_error

In [21]:
# function for predicting next N days prices based on past T days.

def lstm_next7_days(ticker, future_days = 5):
    # sentiment analyser 
    sia = SentimentIntensityAnalyzer()
    end_date = datetime.now().date() 
    print("end_date: ",end_date)

    # use 3 years of data 
    d = timedelta(days = 1095)
    start_date = end_date - d
    print("Start_date: ",start_date)
    print("Stock: ",ticker)

        # fetch data
    StockData = web.get_data_yahoo( ticker + ".NS", start = (start_date), end = (end_date), interval = 'd')

    #==========news extraction start===========================================

    dates_list = list(StockData.index)

        # initialization
    page_no = 1
    StockData['news'] = 0
    StockData['sentiment'] = 0
        # looping over dates 
    for date_counter in range(len(dates_list)):

        ip_date = dates_list[date_counter]
        ip_date = datetime.strftime(ip_date, "%d-%m-%Y")

            # max 4 pages are scraped
        for page_no in range(1,4):
            url = "https://www.business-standard.com/advance-search?advance=Y&type=print-media&print_date=" + ip_date  \
                        + "&itemsPerPage=19&page=" + str(page_no)
            request = requests.get(url)
            parser = BeautifulSoup(request.text, "html.parser")

                # ulink class filtering, link filtering, h2 text, p text capturing
            for ulink in parser.find_all('ul',{'class': 'listing'}): 
                for link in ulink.find_all("li"):
                    if ticker in (link.find("h2")).text:
                        #print((link.find("h2")).text)
                        l= list(link)
                        ptext = (str(l[-4]))[3:-5]
                        #print(ptext) 
                            # news column for the day updated in df
                        StockData.iloc[date_counter,6] = ptext
                            # sentiment scores obtaining
                            # sentiment column for the day updated in df
                            # if no articles for the day then score set to zero else sentimentanalyser score calculated
                        news_content = StockData.iloc[date_counter,6]
                        if news_content == 0:
                            StockData.iloc[date_counter,7] = 0
                        else:
                            sentiment = sia.polarity_scores(news_content)['compound']
                            StockData.iloc[date_counter,7] = sentiment
                            #print("sentiment:",sentiment)

    #==========news extraction end===========================================    


        # Extracting the closing prices of each day
    FullData = StockData[['Close']].values   
    FullData2 = StockData[['Close', 'sentiment']].values      ## updated 'sentiment' added

    # data nomalization    
    sc=MinMaxScaler()
    DataScaler = sc.fit(FullData)
    X = DataScaler.transform(FullData)
    X = pd.DataFrame(X, columns = ['Close'])
    #print(type(X))
    #print(X)
    X.loc[:,'sentiment'] = FullData2[:,1]
    # saving the dataframe as csv for offline testing
    X.to_csv("Stock_price_sentiment" + str(time.time()) + ".csv")
    X = X.to_numpy() # or X.values
    #print("=========")
    #print(X)
    #print("=========")
    #print(X[:,0])
    #print("=========")
    #print(X[:,1])


    # Multi step data preparation

        # split into samples
    X_samples = list()
    y_samples = list()

    NumerOfRows = len(X)
    TimeSteps= 10  # next few day's Price Prediction is based on last "how many past day's prices"
    FutureTimeSteps= future_days # How many days in future you want to predict the prices

        # Iterate thru the values to create combinations
    for i in range(TimeSteps , NumerOfRows-FutureTimeSteps , 1):
        x_sample = X[i-TimeSteps:i]
        y_sample = X[:,0][i:i+FutureTimeSteps]
        X_samples.append(x_sample)
        y_samples.append(y_sample)

        # Reshape the Input as a 3D (samples, Time Steps, Features)
    X_data=np.array(X_samples)
        #X_data=X_data.reshape(X_data.shape[0],X_data.shape[1], 1)     ##======= commented
    X_data=X_data.reshape(X_data.shape[0],X_data.shape[1], 2)      ##=========   added

        # We do not reshape y as a 3D data  as it is supposed to be a single column only
    y_data=np.array(y_samples)

        # Choosing the number of testing data records
    TestingRecords = 30

        # Splitting the data into train and test
    X_train=X_data[:-TestingRecords]
    X_test=X_data[-TestingRecords:]
    y_train=y_data[:-TestingRecords]
    y_test=y_data[-TestingRecords:]
    print("========================================")
    print("X_train", "X_test", "y_train", "y_test")
    print(X_train.shape, X_test.shape, y_train.shape,y_test.shape )
    print("========================================")

        # Defining Input shapes for LSTM
    TimeSteps=X_train.shape[1]
    TotalFeatures=X_train.shape[2]
    print("TotalFeatures",TotalFeatures)

    # Initialising the RNN
    regressor = Sequential()

        # Adding the First input hidden layer and the LSTM layer
        # return_sequences = True, means the output of every time step to be shared with hidden next layer
    regressor.add(LSTM(units = 10, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))


        # Adding the Second hidden layer and the LSTM layer
    regressor.add(LSTM(units = 5, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))

        # Adding the Third hidden layer and the LSTM layer
    regressor.add(LSTM(units = 5, activation = 'relu', return_sequences=False ))


        # Adding the output layer
        # Notice the number of neurons in the dense layer is now the number of future time steps 
        # Based on the number of future days we want to predict
    regressor.add(Dense(units = FutureTimeSteps))

        # Compiling the RNN
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

        # Measuring the time taken by the model to train
    StartTime=time.time()

        # Fitting the RNN to the Training set
    regressor.fit(X_train, y_train, batch_size = 5, epochs = 100, verbose = 0)

    EndTime=time.time()

    # predictions on test data
    predicted_prices = regressor.predict(X_test)
    print("predicted_prices \n", predicted_prices[-3:])
    predicted_prices = DataScaler.inverse_transform(predicted_prices)
    print("predicted_prices: \n", predicted_prices[-3:])

    # Getting the original price values for testing data
    orig=y_test
    orig=DataScaler.inverse_transform(y_test)
    print('\n#### Original Prices ####')
    print(orig[-3:])
    
    # MAPE
    MAPE = mean_absolute_percentage_error(orig, predicted_prices)
    print("MAPE:", MAPE)

    # Making next 5 days price forecast 

    Last10DaysPrices = FullData[-TimeSteps:]
        # Reshaping the data to (-1,1 )because its a single entry
    Last10DaysPrices=Last10DaysPrices.reshape(-1, 1)
        # Scaling the data on the same level on which model was trained
    X_forecast=DataScaler.transform(Last10DaysPrices)
    X_forecast = pd.DataFrame(X_forecast, columns = ['Close'])
    #print(X_forecast)

    X_forecast.loc[:,'sentiment'] = FullData2[-TimeSteps:,1]
    X_forecast = X_forecast.to_numpy() # or X_forecast.values

    NumberofSamples=1
    #TimeSteps= 2 #X_test.shape[0]
    NumberofFeatures= 2 #X_test.shape[1]

        # Reshaping the data as 3D input
    X_forecast = X_forecast.reshape(NumberofSamples,TimeSteps,NumberofFeatures)
    #print("X_forecast ip:", X_forecast)

        # Generating the predictions for next 7 days
    Next7DaysPrice = regressor.predict(X_forecast)

    print("Next7DaysPrice",Next7DaysPrice)

        # Generating the prices in original scale
    Next7DaysPrice = DataScaler.inverse_transform(Next7DaysPrice)

    print("Next7DaysPrice: ", Next7DaysPrice)

In [22]:
lstm_next7_days("TCS", future_days = 5)

end_date:  2022-01-03
Start_date:  2019-01-04
Stock:  TCS
X_train X_test y_train y_test
(695, 10, 2) (30, 10, 2) (695, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.82204187 0.8113527  0.8150477  0.8110453  0.8136455 ]
 [0.8334466  0.822933   0.82645607 0.8225127  0.8247332 ]
 [0.8447974  0.8345282  0.83798414 0.8341031  0.83615404]]
predicted_prices: 
 [[3542.0073 3517.2278 3525.7937 3516.5151 3522.543 ]
 [3568.4458 3544.0735 3552.2405 3543.099  3548.2466]
 [3594.7593 3570.9534 3578.9648 3569.9678 3574.7224]]

#### Original Prices ####
[[3662.69995117 3670.89990234 3696.10009766 3706.55004883 3694.69995117]
 [3670.89990234 3696.10009766 3706.55004883 3694.69995117 3733.75      ]
 [3696.10009766 3706.55004883 3694.69995117 3733.75       3738.35009766]]
MAPE: 0.022248289640869934
Next7DaysPrice [[0.8988317  0.88426906 0.8879785  0.8826811  0.88512194]]
Next7DaysPrice:  [[3720.0217 3686.2627 3694.8618 3682.5813 3688.2397]]


In [24]:
lstm_next7_days("TATASTEEL", future_days = 5)

end_date:  2022-01-03
Start_date:  2019-01-04
Stock:  TATASTEEL
X_train X_test y_train y_test
(695, 10, 2) (30, 10, 2) (695, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.7055557  0.70306426 0.70811    0.71159184 0.7127653 ]
 [0.70278    0.70087665 0.70559746 0.7095966  0.71072316]
 [0.70087713 0.69948745 0.7038876  0.7082576  0.7093921 ]]
predicted_prices: 
 [[1146.7366 1143.5834 1149.9695 1154.3762 1155.8615]
 [1143.2235 1140.8146 1146.7894 1151.851  1153.2769]
 [1140.8152 1139.0563 1144.6254 1150.1562 1151.5922]]

#### Original Prices ####
[[1127.65002441 1115.44995117 1121.80004883 1127.44995117 1116.25      ]
 [1115.44995117 1121.80004883 1127.44995117 1116.25       1101.        ]
 [1121.80004883 1127.44995117 1116.25       1101.         1111.44995117]]
MAPE: 0.05044833617330091
Next7DaysPrice [[0.70056206 0.699988   0.703686   0.708221   0.7096238 ]]
Next7DaysPrice:  [[1140.4164 1139.6898 1144.3702 1150.11   1151.8854]]


In [25]:
lstm_next7_days("TATAPOWER", future_days = 5)

end_date:  2022-01-03
Start_date:  2019-01-04
Stock:  TATAPOWER
X_train X_test y_train y_test
(695, 10, 2) (30, 10, 2) (695, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.8150868  0.82175404 0.83263034 0.8329723  0.8411811 ]
 [0.8330057  0.8397802  0.85083675 0.8515793  0.85984313]
 [0.8523034  0.8592259  0.8704947  0.87155706 0.87992495]]
predicted_prices: 
 [[214.76996 216.30342 218.80498 218.88362 220.77165]
 [218.89131 220.44945 222.99245 223.16324 225.06392]
 [223.32977 224.92195 227.51378 227.75812 229.68274]]

#### Original Prices ####
[[218.05000305 216.80000305 217.3500061  221.6000061  219.44999695]
 [216.80000305 217.3500061  221.6000061  219.44999695 218.3500061 ]
 [217.3500061  221.6000061  219.44999695 218.3500061  220.94999695]]
MAPE: 0.03307766242123448
Next7DaysPrice [[0.8636856  0.87069607 0.8820903  0.8833394  0.89176947]]
Next7DaysPrice:  [[225.94768 227.56009 230.18076 230.46805 232.40697]]


In [26]:
lstm_next7_days("TATAMOTORS", future_days = 5)

end_date:  2022-01-03
Start_date:  2019-01-04
Stock:  TATAMOTORS
X_train X_test y_train y_test
(695, 10, 2) (30, 10, 2) (695, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.8917733  0.8604656  0.8954479  0.87842035 0.8613202 ]
 [0.909551   0.8904964  0.9106312  0.9054261  0.8961243 ]
 [0.8838924  0.88106906 0.88232774 0.8961906  0.89293003]]
predicted_prices: 
 [[479.84085 465.28748 481.549   473.6337  465.68472]
 [488.1048  479.24725 488.6069  486.18735 481.8634 ]
 [476.1774  474.86496 475.45007 481.89423 480.37854]]

#### Original Prices ####
[[472.3500061  467.6000061  471.1499939  480.20001221 476.        ]
 [467.6000061  471.1499939  480.20001221 476.         470.3999939 ]
 [471.1499939  480.20001221 476.         470.3999939  482.3999939 ]]
MAPE: 0.040699328471055896
Next7DaysPrice [[0.960632  0.9436139 0.9614477 0.9545895 0.9507486]]
Next7DaysPrice:  [[511.84982 503.93893 512.229   509.04092 507.25552]]


In [27]:
lstm_next7_days("SBIN", future_days = 5)

end_date:  2022-01-03
Start_date:  2019-01-04
Stock:  SBIN
X_train X_test y_train y_test
(695, 10, 2) (30, 10, 2) (695, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.755264   0.75589836 0.75769067 0.7595791  0.7610849 ]
 [0.78293383 0.78492284 0.78647923 0.7902361  0.7900057 ]
 [0.78061694 0.78273714 0.78426564 0.7882023  0.7878069 ]]
predicted_prices: 
 [[437.54822 437.78903 438.4694  439.18625 439.75784]
 [448.0517  448.80673 449.39755 450.82364 450.73618]
 [447.17218 447.97705 448.55725 450.0516  449.90152]]

#### Original Prices ####
[[461.79998779 456.95001221 458.04998779 461.20001221 454.3999939 ]
 [456.95001221 458.04998779 461.20001221 454.3999939  451.70001221]
 [458.04998779 461.20001221 454.3999939  451.70001221 460.45001221]]
MAPE: 0.030128696250733493
Next7DaysPrice [[0.8176877  0.8221319  0.8232448  0.830391   0.82701004]]
Next7DaysPrice:  [[461.24426 462.93127 463.35376 466.0664  464.78302]]


In [28]:
lstm_next7_days("BANKBARODA", future_days = 5)

end_date:  2022-01-03
Start_date:  2019-01-04
Stock:  BANKBARODA
X_train X_test y_train y_test
(695, 10, 2) (30, 10, 2) (695, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.4327189  0.42120883 0.41590166 0.4324708  0.42734185]
 [0.42055333 0.41152176 0.4082359  0.42012447 0.4184998 ]
 [0.4116214  0.4062574  0.40195137 0.4101514  0.4119491 ]]
predicted_prices: 
 [[81.69749  80.49526  79.940926 81.67158  81.13585 ]
 [80.426796 79.483444 79.14024  80.382    80.2123  ]
 [79.49385  78.933586 78.48382  79.34031  79.52808 ]]

#### Original Prices ####
[[81.19999695 79.44999695 79.84999847 80.90000153 80.94999695]
 [79.44999695 79.84999847 80.90000153 80.94999695 80.15000153]
 [79.84999847 80.90000153 80.94999695 80.15000153 81.94999695]]
MAPE: 0.05101029440100198
Next7DaysPrice [[0.4193339  0.41701046 0.42169613 0.4142519  0.4235878 ]]
Next7DaysPrice:  [[80.29942  80.05674  80.54616  79.76861  80.743744]]


In [29]:
lstm_next7_days("EQUITASBNK", future_days = 5)

end_date:  2022-01-03
Start_date:  2019-01-04
Stock:  EQUITASBNK
X_train X_test y_train y_test
(246, 10, 2) (30, 10, 2) (246, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.5814633  0.57604027 0.57929194 0.58560616 0.58599126]
 [0.5814633  0.57604027 0.57929194 0.58560616 0.58599126]
 [0.5814633  0.57604027 0.57929194 0.58560616 0.58599126]]
predicted_prices: 
 [[54.30951  54.107502 54.228626 54.46383  54.478176]
 [54.30951  54.107502 54.228626 54.46383  54.478176]
 [54.30951  54.107502 54.228626 54.46383  54.478176]]

#### Original Prices ####
[[60.         60.15000153 59.95000076 58.84999847 61.        ]
 [60.15000153 59.95000076 58.84999847 61.         59.79999924]
 [59.95000076 58.84999847 61.         59.79999924 59.5       ]]
MAPE: 0.11667210949591925
Next7DaysPrice [[0.5814633  0.57604027 0.57929194 0.58560616 0.58599126]]
Next7DaysPrice:  [[54.30951  54.107502 54.228626 54.46383  54.478176]]


In [31]:
lstm_next7_days("ASIANPAINT", future_days = 5)

end_date:  2022-01-03
Start_date:  2019-01-04
Stock:  ASIANPAINT
X_train X_test y_train y_test
(695, 10, 2) (30, 10, 2) (695, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.92293704 0.9294195  0.91858554 0.9164728  0.91026706]
 [0.93132347 0.93869823 0.929365   0.9242811  0.91866714]
 [0.93023926 0.93761575 0.9285137  0.9231301  0.91766286]]
predicted_prices: 
 [[3283.3616 3297.2612 3274.031  3269.501  3256.1948]
 [3301.344  3317.1567 3297.1445 3286.2437 3274.2063]
 [3299.019  3314.8357 3295.319  3283.7756 3272.0527]]

#### Original Prices ####
[[3267.89990234 3284.80004883 3272.39990234 3368.19995117 3367.44995117]
 [3284.80004883 3272.39990234 3368.19995117 3367.44995117 3365.69995117]
 [3272.39990234 3368.19995117 3367.44995117 3365.69995117 3382.94995117]]
MAPE: 0.021291149005469027
Next7DaysPrice [[0.967259   0.983491   0.97574407 0.96049213 0.9557357 ]]
Next7DaysPrice:  [[3378.3967 3413.2014 3396.5906 3363.8872 3353.6885]]


In [32]:
lstm_next7_days("ONGC", future_days = 5)

end_date:  2022-01-03
Start_date:  2019-01-04
Stock:  ONGC
X_train X_test y_train y_test
(695, 10, 2) (30, 10, 2) (695, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.65324056 0.6347881  0.6375015  0.6371608  0.6450918 ]
 [0.6662766  0.64876485 0.6512643  0.6510541  0.6584674 ]
 [0.66370714 0.6437799  0.6467228  0.6461915  0.65479994]]
predicted_prices: 
 [[136.42914 134.27022 134.58768 134.5478  135.47574]
 [137.95436 135.90549 136.19792 136.17332 137.04068]
 [137.65373 135.32224 135.66656 135.6044  136.61159]]

#### Original Prices ####
[[140.5       138.5       137.3500061 139.5       138.8999939]
 [138.5       137.3500061 139.5       138.8999939 140.5      ]
 [137.3500061 139.5       138.8999939 140.5       142.3999939]]
MAPE: 0.03340375150153091
Next7DaysPrice [[0.6975162  0.6829254  0.6847925  0.68498313 0.69082904]]
Next7DaysPrice:  [[141.60939 139.90228 140.12073 140.14302 140.827  ]]


In [33]:
lstm_next7_days("AMRUTANJAN", future_days = 5)

end_date:  2022-01-03
Start_date:  2019-01-04
Stock:  AMRUTANJAN
X_train X_test y_train y_test
(695, 10, 2) (30, 10, 2) (695, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.8594028  0.85818356 0.86783904 0.8693957  0.87227476]
 [0.86380154 0.8625691  0.87095654 0.8725097  0.87552047]
 [0.86386496 0.86263055 0.87084043 0.87239224 0.87542033]]
predicted_prices: 
 [[890.3912  889.5096  896.49097 897.6165  899.69824]
 [893.57166 892.6806  898.7451  899.86816 902.04504]
 [893.6175  892.725   898.6612  899.78314 901.97266]]

#### Original Prices ####
[[901.45001221 898.70001221 915.45001221 915.54998779 908.        ]
 [898.70001221 915.45001221 915.54998779 908.         935.65002441]
 [915.45001221 915.54998779 908.         935.65002441 936.5       ]]
MAPE: 0.017069381620804444
Next7DaysPrice [[0.89664894 0.89542264 0.90375775 0.90536374 0.90844303]]
Next7DaysPrice:  [[917.32196 916.4353  922.462   923.6232  925.8497 ]]


In [34]:
lstm_next7_days("WIPRO", future_days = 5)

end_date:  2022-01-04
Start_date:  2019-01-05
Stock:  WIPRO
X_train X_test y_train y_test
(694, 10, 2) (30, 10, 2) (694, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.93160015 0.93592906 0.93796015 0.9407525  0.94018394]
 [0.93672276 0.9410723  0.9431134  0.9459217  0.94535553]
 [0.9436557  0.9480332  0.95008785 0.9529178  0.95235485]]
predicted_prices: 
 [[680.64575 683.05414 684.1841  685.7376  685.4213 ]
 [683.49567 685.9156  687.05115 688.6136  688.2985 ]
 [687.35284 689.78827 690.9314  692.5058  692.1926 ]]

#### Original Prices ####
[[694.54998779 698.45001221 699.59997559 705.45001221 703.70001221]
 [698.45001221 699.59997559 705.45001221 703.70001221 714.09997559]
 [699.59997559 705.45001221 703.70001221 714.09997559 715.34997559]]
MAPE: 0.026604092204366252
Next7DaysPrice [[0.9720027  0.97649455 0.9786045  0.98152286 0.9809731 ]]
Next7DaysPrice:  [[703.1237  705.62274 706.79663 708.4202  708.1144 ]]


In [1]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)
import pandas_datareader.data as web
#import datetime 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time 

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# To remove the scientific notation from numpy arrays
np.set_printoptions(suppress=True)
# Feature Scaling for fast training of neural networks
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

# =========================================================
import sys
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from math import isnan
# ============================================================


def lstm_next7_days(ticker, future_days = 5):

    sia = SentimentIntensityAnalyzer()
    end_date = datetime.now().date() 
    print("end_date: ",end_date)

    # use 3 years of data 
    d = timedelta(days = 1095)
    #d = timedelta(days = 15)
    start_date = end_date - d
    print("Start_date: ",start_date)
    print("Stock: ",ticker)

        # fetch data
    StockData = web.get_data_yahoo( ticker + ".NS", start = (start_date), end = (end_date), interval = 'd')

    #==========news extraction start===========================================

    dates_list = list(StockData.index)

        # initialization
    page_no = 1
    StockData['news'] = 0
    StockData['sentiment'] = 0
        # looping over dates 
    for date_counter in range(len(dates_list)):

        ip_date = dates_list[date_counter]
        ip_date = datetime.strftime(ip_date, "%d-%m-%Y")

            # max 4 pages are scraped
        for page_no in range(1,4):
            url = "https://www.business-standard.com/advance-search?advance=Y&type=print-media&print_date=" + ip_date  \
                        + "&itemsPerPage=19&page=" + str(page_no)
            request = requests.get(url)
            parser = BeautifulSoup(request.text, "html.parser")

                # ulink class filtering, link filtering, h2 text, p text capturing
            for ulink in parser.find_all('ul',{'class': 'listing'}): 
                for link in ulink.find_all("li"):
                    if ticker in (link.find("h2")).text:
                        #print((link.find("h2")).text)
                        l= list(link)
                        ptext = (str(l[-4]))[3:-5]
                        #print(ptext) 
                            # news column for the day updated in df
                        StockData.iloc[date_counter,6] = ptext
                            # sentiment scores obtaining
                            # sentiment column for the day updated in df
                            # if no articles for the day then score set to zero else sentimentanalyser score calculated
                        news_content = StockData.iloc[date_counter,6]
                        if news_content == 0:
                            StockData.iloc[date_counter,7] = 0
                        else:
                            sentiment = sia.polarity_scores(news_content)['compound']
                            StockData.iloc[date_counter,7] = sentiment
                            #print("sentiment:",sentiment)

    #==========news extraction end===========================================    


        # Extracting the closing prices of each day
    FullData = StockData[['Close']].values   
    FullData2 = StockData[['Close', 'sentiment']].values      ## updated 'sentiment' added

        # Choosing between Standardization or normalization
        #sc = StandardScaler()
    sc=MinMaxScaler()
    DataScaler = sc.fit(FullData)
    X = DataScaler.transform(FullData)
    X = pd.DataFrame(X, columns = ['Close'])
    #print(type(X))
    #print(X)
    X.loc[:,'sentiment'] = FullData2[:,1]
    # saving the dataframe as csv for offline repeated testing
    X.to_csv("Stock_price_sentiment" + str(time.time()) + ".csv")
    X = X.to_numpy() # or X.values
    #print("=========")
    #print(X)
    #print("=========")
    #print(X[:,0])
    #print("=========")
    #print(X[:,1])


    # Multi step data preparation

        # split into samples
    X_samples = list()
    y_samples = list()

    NumerOfRows = len(X)
    TimeSteps= 10  # next few day's Price Prediction is based on last "how many past day's prices"
    FutureTimeSteps= future_days # How many days in future you want to predict the prices

        # Iterate thru the values to create combinations
    for i in range(TimeSteps , NumerOfRows-FutureTimeSteps , 1):
        x_sample = X[i-TimeSteps:i]
        y_sample = X[:,0][i:i+FutureTimeSteps]
        X_samples.append(x_sample)
        y_samples.append(y_sample)

        # Reshape the Input as a 3D (samples, Time Steps, Features)
    X_data=np.array(X_samples)
        #X_data=X_data.reshape(X_data.shape[0],X_data.shape[1], 1)     ##======= commented
    X_data=X_data.reshape(X_data.shape[0],X_data.shape[1], 2)      ##=========   added

        # We do not reshape y as a 3D data  as it is supposed to be a single column only
    y_data=np.array(y_samples)

        # Choosing the number of testing data records
    TestingRecords = 30

        # Splitting the data into train and test
    X_train=X_data[:-TestingRecords]
    X_test=X_data[-TestingRecords:]
    y_train=y_data[:-TestingRecords]
    y_test=y_data[-TestingRecords:]
    print("========================================")
    print("X_train", "X_test", "y_train", "y_test")
    print(X_train.shape, X_test.shape, y_train.shape,y_test.shape )
    print("========================================")

        # Defining Input shapes for LSTM
    TimeSteps=X_train.shape[1]
    TotalFeatures=X_train.shape[2]
    print("TotalFeatures",TotalFeatures)

    # Initialising the RNN
    regressor = Sequential()

        # Adding the First input hidden layer and the LSTM layer
        # return_sequences = True, means the output of every time step to be shared with hidden next layer
    regressor.add(LSTM(units = 10, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))


        # Adding the Second hidden layer and the LSTM layer
    regressor.add(LSTM(units = 5, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))

        # Adding the Third hidden layer and the LSTM layer
    regressor.add(LSTM(units = 5, activation = 'relu', return_sequences=False ))


        # Adding the output layer
        # Notice the number of neurons in the dense layer is now the number of future time steps 
        # Based on the number of future days we want to predict
    regressor.add(Dense(units = FutureTimeSteps))

        # Compiling the RNN
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

        # Measuring the time taken by the model to train
    StartTime=time.time()

        # Fitting the RNN to the Training set
    regressor.fit(X_train, y_train, batch_size = 5, epochs = 100, verbose = 0)

    EndTime=time.time()

    # predictions on test data
    predicted_prices = regressor.predict(X_test)
    print("predicted_prices \n", predicted_prices[-3:])
    predicted_prices = DataScaler.inverse_transform(predicted_prices)
    print("predicted_prices: \n", predicted_prices[-3:])

    # Getting the original price values for testing data
    orig=y_test
    orig=DataScaler.inverse_transform(y_test)
    print('\n#### Original Prices ####')
    print(orig[-3:])
    
    # MAPE
    MAPE = mean_absolute_percentage_error(orig, predicted_prices)
    print("MAPE:", MAPE)

    # Making next 5 days price forecast 

    Last10DaysPrices = FullData[-TimeSteps:]
        # Reshaping the data to (-1,1 )because its a single entry
    Last10DaysPrices=Last10DaysPrices.reshape(-1, 1)
        # Scaling the data on the same level on which model was trained
    X_forecast=DataScaler.transform(Last10DaysPrices)
    X_forecast = pd.DataFrame(X_forecast, columns = ['Close'])
    #print(X_forecast)

    X_forecast.loc[:,'sentiment'] = FullData2[-TimeSteps:,1]
    X_forecast = X_forecast.to_numpy() # or X_forecast.values

    NumberofSamples=1
    #TimeSteps= 2 #X_test.shape[0]
    NumberofFeatures= 2 #X_test.shape[1]

        # Reshaping the data as 3D input
    X_forecast = X_forecast.reshape(NumberofSamples,TimeSteps,NumberofFeatures)
    #print("X_forecast ip:", X_forecast)

        # Generating the predictions for next 7 days
    Next7DaysPrice = regressor.predict(X_forecast)

    print("Next7DaysPrice",Next7DaysPrice)

        # Generating the prices in original scale
    Next7DaysPrice = DataScaler.inverse_transform(Next7DaysPrice)

    print("Next7DaysPrice: ", Next7DaysPrice)

In [2]:
lstm_next7_days("ITC", future_days = 5)

end_date:  2022-01-04
Start_date:  2019-01-05
Stock:  ITC
X_train X_test y_train y_test
(694, 10, 2) (30, 10, 2) (694, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.38408136 0.38160443 0.38330203 0.3868352  0.3911156 ]
 [0.41188323 0.39835864 0.40634292 0.40546614 0.4078158 ]
 [0.42257243 0.40370703 0.41475117 0.4118879  0.41348982]]
predicted_prices: 
 [[208.68382 208.28764 208.55916 209.12428 209.80894]
 [213.13074 210.96747 212.24455 212.10431 212.48015]
 [214.84047 211.82295 213.58946 213.13148 213.38771]]

#### Original Prices ####
[[217.30000305 218.         217.55000305 220.05000305 216.75      ]
 [218.         217.55000305 220.05000305 216.75       216.        ]
 [217.55000305 220.05000305 216.75       216.         218.05000305]]
MAPE: 0.0268268892379979
Next7DaysPrice [[0.43833363 0.42171448 0.43130407 0.42831832 0.42901397]]
Next7DaysPrice:  [[217.36148 214.70323 216.23709 215.75952 215.8708 ]]


In [3]:
lstm_next7_days("DABUR", future_days = 5)

end_date:  2022-01-04
Start_date:  2019-01-05
Stock:  DABUR
X_train X_test y_train y_test
(694, 10, 2) (30, 10, 2) (694, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.68214643 0.6928235  0.6929372  0.69260126 0.69863737]
 [0.701998   0.7114582  0.71051455 0.71254116 0.71673715]
 [0.6861869  0.6961897  0.69550353 0.69687265 0.7016983 ]]
predicted_prices: 
 [[560.8507  563.978   564.0113  563.9129  565.6809 ]
 [566.6652  569.43616 569.1597  569.7533  570.9823 ]
 [562.0342  564.964   564.763   565.164   566.57745]]

#### Original Prices ####
[[569.45001221 563.70001221 561.45001221 562.45001221 566.59997559]
 [563.70001221 561.45001221 562.45001221 566.59997559 571.54998779]
 [561.45001221 562.45001221 566.59997559 571.54998779 580.04998779]]
MAPE: 0.01300062179667567
Next7DaysPrice [[0.7333859 0.7389678 0.7369153 0.7430655 0.7434416]]
Next7DaysPrice:  [[575.85876 577.49365 576.8925  578.6939  578.8041 ]]


In [4]:
lstm_next7_days("NMDC", future_days = 5)

end_date:  2022-01-04
Start_date:  2019-01-05
Stock:  NMDC
X_train X_test y_train y_test
(695, 10, 2) (30, 10, 2) (695, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.5231907  0.53352135 0.5269335  0.5328368  0.53004   ]
 [0.51344913 0.5225437  0.5181962  0.52390546 0.52397096]
 [0.51077175 0.52001023 0.5160539  0.5218378  0.5224451 ]]
predicted_prices: 
 [[136.24437 137.68964 136.76799 137.59387 137.20259]
 [134.88153 136.15387 135.54565 136.34438 136.35353]
 [134.50697 135.79942 135.24594 136.0551  136.14006]]

#### Original Prices ####
[[132.6000061  132.5        132.69999695 131.3999939  131.3999939 ]
 [132.5        132.69999695 131.3999939  131.3999939  133.30000305]
 [132.69999695 131.3999939  131.3999939  133.30000305 135.30000305]]
MAPE: 0.040680533922261564
Next7DaysPrice [[0.5271243  0.53851724 0.5337906  0.54015696 0.54013073]]
Next7DaysPrice:  [[136.7947  138.38857 137.7273  138.61795 138.61429]]


In [6]:
lstm_next7_days("PFIZER", future_days = 5)

end_date:  2022-01-06
Start_date:  2019-01-07
Stock:  PFIZER
X_train X_test y_train y_test
(697, 10, 2) (30, 10, 2) (697, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.6778953  0.68572676 0.6855668  0.6908181  0.687625  ]
 [0.68441415 0.6936459  0.69048923 0.69385076 0.6919452 ]
 [0.6841524  0.69320875 0.69100535 0.6955336  0.6929962 ]]
predicted_prices: 
 [[4994.6777 5021.6187 5021.0684 5039.1333 5028.1484]
 [5017.103  5048.861  5038.002  5049.566  5043.0107]
 [5016.2026 5047.3574 5039.7773 5055.355  5046.626 ]]

#### Original Prices ####
[[5046.95019531 5046.85009766 5029.14990234 5055.29980469 5100.60009766]
 [5046.85009766 5029.14990234 5055.29980469 5100.60009766 5064.70019531]
 [5029.14990234 5055.29980469 5100.60009766 5064.70019531 5063.79980469]]
MAPE: 0.02314800477535615
Next7DaysPrice [[0.6829574  0.6927377  0.6903712  0.69600993 0.6931539 ]]
Next7DaysPrice:  [[5012.092  5045.737  5037.5957 5056.9937 5047.1685]]


In [7]:
lstm_next7_days("LUPIN", future_days = 5)

end_date:  2022-01-06
Start_date:  2019-01-07
Stock:  LUPIN
X_train X_test y_train y_test
(697, 10, 2) (30, 10, 2) (697, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.52959096 0.5301738  0.5328245  0.5243549  0.52794516]
 [0.53181565 0.53246665 0.53513813 0.5264476  0.5301148 ]
 [0.54759985 0.54816186 0.5507499  0.5418094  0.54543865]]
predicted_prices: 
 [[918.3635  918.771   920.62427 914.70276 917.2128 ]
 [919.9189  920.374   922.2418  916.1659  918.7298 ]
 [930.95447 931.34735 933.1568  926.90607 929.4434 ]]

#### Original Prices ####
[[916.59997559 929.59997559 937.84997559 950.75       945.25      ]
 [929.59997559 937.84997559 950.75       945.25       932.34997559]
 [937.84997559 950.75       945.25       932.34997559 932.04998779]]
MAPE: 0.02430501795715289
Next7DaysPrice [[0.5626067  0.5628841  0.56531227 0.5565948  0.5599836 ]]
Next7DaysPrice:  [[941.4465  941.6404  943.3381  937.2433  939.61255]]


In [8]:
lstm_next7_days("ALKEM", future_days = 5)

end_date:  2022-01-06
Start_date:  2019-01-07
Stock:  ALKEM
X_train X_test y_train y_test
(697, 10, 2) (30, 10, 2) (697, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.77321744 0.77011096 0.77056855 0.76128095 0.76417494]
 [0.77970994 0.7763678  0.7771895  0.76804644 0.770347  ]
 [0.78825086 0.784106   0.7859085  0.7761159  0.7777447 ]]
predicted_prices: 
 [[3473.7668 3466.5159 3467.5842 3445.906  3452.6606]
 [3488.921  3481.12   3483.038  3461.6973 3467.067 ]
 [3508.8564 3499.182  3503.3892 3480.532  3484.334 ]]

#### Original Prices ####
[[3485.05004883 3517.75       3624.         3624.05004883 3641.10009766]
 [3517.75       3624.         3624.05004883 3641.10009766 3613.80004883]
 [3624.         3624.05004883 3641.10009766 3613.80004883 3602.80004883]]
MAPE: 0.02221570967239186
Next7DaysPrice [[0.8519872 0.8475052 0.8485068 0.8374937 0.8398435]]
Next7DaysPrice:  [[3657.6233 3647.1619 3649.4998 3623.7942 3629.2788]]


In [10]:
lstm_next7_days("CIPLA", future_days = 5)

end_date:  2022-01-06
Start_date:  2019-01-07
Stock:  CIPLA
X_train X_test y_train y_test
(697, 10, 2) (30, 10, 2) (697, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.88778967 0.88882035 0.88174874 0.8771491  0.8842775 ]
 [0.8934811  0.894338   0.88688064 0.88155633 0.88928634]
 [0.89751637 0.8982352  0.8905257  0.88465893 0.8928529 ]]
predicted_prices: 
 [[918.8263  919.458   915.12384 912.3047  916.6737 ]
 [922.3146  922.8397  918.2691  915.00586 919.7436 ]
 [924.7878  925.22833 920.5032  916.9074  921.9295 ]]

#### Original Prices ####
[[933.29998779 935.54998779 952.75       944.09997559 930.5       ]
 [935.54998779 952.75       944.09997559 930.5        924.25      ]
 [952.75       944.09997559 930.5        924.25       928.54998779]]
MAPE: 0.024462359260645654
Next7DaysPrice [[0.892653   0.8940871  0.8878585  0.88495487 0.8906742 ]]
Next7DaysPrice:  [[921.807   922.686   918.86847 917.08887 920.59424]]


In [11]:
lstm_next7_days("DLF", future_days = 5)

end_date:  2022-01-06
Start_date:  2019-01-07
Stock:  DLF
X_train X_test y_train y_test
(697, 10, 2) (30, 10, 2) (697, 5) (30, 5)
TotalFeatures 2
predicted_prices 
 [[0.8409455  0.8477223  0.8676449  0.854663   0.8591261 ]
 [0.8462583  0.84936637 0.86587787 0.86257464 0.8599544 ]
 [0.8494155  0.8497552  0.86217207 0.8672636  0.85867506]]
predicted_prices: 
 [[391.0799  393.25797 399.6611  395.4887  396.92316]
 [392.78745 393.78635 399.09317 398.0315  397.18936]
 [393.80215 393.91135 397.9021  399.53854 396.77817]]

#### Original Prices ####
[[385.79998779 385.70001221 379.20001221 390.45001221 394.75      ]
 [385.70001221 379.20001221 390.45001221 394.75       394.45001221]
 [379.20001221 390.45001221 394.75       394.45001221 402.04998779]]
MAPE: 0.04491286524341144
Next7DaysPrice [[0.8825996  0.8845956  0.89165485 0.9050513  0.8924846 ]]
Next7DaysPrice:  [[404.46753 405.10904 407.3779  411.6835  407.64456]]


In [12]:
# all 19 stocks
# avg MAPE
list_MAPE = [2.22, 5.04, 3.30, 4.06, 3.01, 5.10, 11.66, 2.12, 3.34, 1.70, 2.66, 2.68, 1.30, 4.06, 2.31, 2.43, 2.22, 2.44, 4.49]
np.mean(list_MAPE)

3.4810526315789474

In [13]:
np.median(list_MAPE)

2.68

In [14]:
np.std(list_MAPE)

2.197452819053633

In [15]:
# 10 stocks
# avg MAPE
list_MAPE = [2.22, 5.04, 3.30, 4.06, 3.01, 11.66, 2.12, 3.34, 1.70, 2.66]
np.mean(list_MAPE)

3.911

In [16]:
np.std(list_MAPE)

2.7454633488720988